In [1]:
!pip install pytesseract opencv-python Pillow fpdf python-docx langdetect googletrans==4.0.0-rc1 transformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=38d1d2eeeb5c34264c13515b709d341177797916ea25ca000dc9206fb8d2ad4b
  

In [2]:
!apt-get update
!apt-get install -y tesseract-ocr


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,563 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,517 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,631

In [3]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'


In [9]:
from google.colab import files
import os

# Create directory for images
os.makedirs("images", exist_ok=True)

# Upload images to the directory
uploaded = files.upload()
for filename in uploaded.keys():
    os.rename(filename, os.path.join("images", filename))

image_folder = "image"

Saving uploaded_image.jpg to uploaded_image.jpg


In [10]:
def preprocess_image(img_path, preprocess_type="grayscale"):
    img = cv2.imread(img_path)
    if preprocess_type == "grayscale":
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    elif preprocess_type == "threshold":
        _, img = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 128, 255, cv2.THRESH_BINARY)
    elif preprocess_type == "edges":
        img = cv2.Canny(img, 100, 200)
    cv2.imwrite("processed_image.jpg", img)
    return "processed_image.jpg"


In [11]:
import os
import pytesseract
from PIL import Image

# Folder containing uploaded images
image_folder = "images"

# Check if the folder exists and contains valid files
if not os.path.exists(image_folder):
    print(f"Directory '{image_folder}' does not exist.")
else:
    print(f"Processing images in '{image_folder}'...")

    for image_file in os.listdir(image_folder):
        # Process only image files
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            file_path = os.path.join(image_folder, image_file)

            # Load and process each image
            print(f"Processing file: {image_file}")
            text = pytesseract.image_to_string(Image.open(file_path))
            print(f"Extracted Text from {image_file}:\n{text}\n")


Processing images in 'images'...
Processing file: uploaded_image.jpg
Extracted Text from uploaded_image.jpg:
SHADOWING PRACTICE 08

The city has contracted with Acme Construction to improve certain
roads in your neighborhood. Work will commence on June 3rd, as
shown in the project schedule. This work will require road closures at
certain times throughout the project. We will be starting work

at 7:00 AM and will finish around 5:00 PM. If you are within the active
work zone limits and park your car in your driveway at night, please
be sure to move it prior to the 7:00 AM start time. We understand that
this may be a short-term inconvenience and we hope to work together
with you in a professional manner for the long-term improvement of
your neighborhood.




In [7]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

DetectorFactory.seed = 0  # For consistent results

# Folder containing uploaded images
image_folder = "images"

# Check if the folder exists and contains valid files
if not os.path.exists(image_folder):
    print(f"Directory '{image_folder}' does not exist.")
else:
    print(f"Processing images in '{image_folder}'...")

    for image_file in os.listdir(image_folder):
        # Process only image files
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            file_path = os.path.join(image_folder, image_file)

            # Load and process each image
            print(f"Processing file: {image_file}")
            text = pytesseract.image_to_string(Image.open(file_path))
            print(f"Extracted Text from {image_file}:\n{text}\n")

            # Detect language
            try:
                language = detect(text)
                print(f"Detected Language: {language}\n")
            except LangDetectException:
                print("Could not detect language. The text might be empty or invalid.\n")


Processing images in 'images'...


In [12]:
from fpdf import FPDF

def save_to_pdf(text, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('Arial', 'B', 12)
    pdf.multi_cell(0, 10, text)
    pdf.output(filename)

save_to_pdf(text, "output.pdf")


In [13]:
from docx import Document

def save_to_docx(text, filename):
    doc = Document()
    doc.add_paragraph(text)
    doc.save(filename)

save_to_docx('text', "output.docx")
print("Document saved successfully!")


Document saved successfully!


In [14]:
from google.colab import files

files.download("output.pdf")  # For PDF
files.download("output.docx")  # For DOCX


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>